In [1]:
import os

#### 1 加载数据

In [2]:
train_texts  = open ('train_contents.txt', encoding = 'utf-8').read().split ('\n')
train_labels = open ('train_labels.txt', encoding = 'utf-8').read().split ('\n')
test_texts   = open ("test_contents.txt", encoding = 'utf-8').read().split ('\n')
test_labels  = open ("test_labels.txt", encoding = 'utf-8').read().split('\n')

all_texts  = train_texts + test_texts
all_labels = train_labels + test_labels

In [8]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 200
VALIDATION_SPLIT = 0.16
TEST_SPLIT = 0.2

#### 2 进行序列转换

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts (all_texts)
sequences = tokenizer.texts_to_sequences (all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 65604 unique tokens.


In [10]:
data = pad_sequences (sequences, maxlen = MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray (all_labels))
print ("shape of data is :", data.shape)
print ("shape of label is :", labels.shape)

shape of data is : (21924, 100)
shape of label is : (21924, 12)


#### 3 随机数据并切分数据

In [11]:
indices = np.arange (data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]

#### 4 搭建CNN网络

In [12]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential

In [20]:
model = Sequential()
model.add (Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length = MAX_SEQUENCE_LENGTH))
model.add (Dropout(0.2))
model.add (Conv1D(250, 3, padding = 'valid', activation = 'relu', strides = 1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 200)          13121000  
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 200)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 250)           150250    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               1600200   
_________________________________________________________________
dense_6 (Dense)              (None, 12)                2412      
Total para

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
model.save('cnn.h5')

Train on 14031 samples, validate on 3508 samples
Epoch 1/2
14031/14031 [==============================] - 72s 5ms/step - loss: 1.2813 - acc: 0.5675 - val_loss: 0.6464 - val_acc: 0.7928
Epoch 2/2
14031/14031 [==============================] - 77s 5ms/step - loss: 0.3774 - acc: 0.8915 - val_loss: 0.5472 - val_acc: 0.8426


In [19]:
print (model.evaluate(x_test, y_test))

4385/4385 [==============================] - 6s 1ms/step
[0.5585867543312951, 0.8223489167616875]


In [23]:
model = Sequential()
model.add (Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length = MAX_SEQUENCE_LENGTH))
model.add (Dropout(0.2))
model.add (Conv1D(256, 3, padding = 'same', activation = 'relu', strides = 1))
model.add(MaxPooling1D(2))
model.add (Conv1D(128, 3, padding = 'valid', activation = 'relu', strides = 1))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=128)

print (model.evaluate(x_test, y_test))

Train on 14031 samples, validate on 3508 samples
Epoch 1/5
14031/14031 [==============================] - 93s 7ms/step - loss: 1.2837 - acc: 0.5521 - val_loss: 0.8196 - val_acc: 0.7175
Epoch 2/5
14031/14031 [==============================] - 86s 6ms/step - loss: 0.4331 - acc: 0.8602 - val_loss: 0.5575 - val_acc: 0.8310
Epoch 3/5
14031/14031 [==============================] - 85s 6ms/step - loss: 0.1795 - acc: 0.9456 - val_loss: 0.6363 - val_acc: 0.8446
Epoch 4/5
14031/14031 [==============================] - 86s 6ms/step - loss: 0.0667 - acc: 0.9794 - val_loss: 0.8739 - val_acc: 0.8381
Epoch 5/5
4385/4385 [==============================] - 7s 2ms/step
[0.8356609413111444, 0.8581527936145952]


In [24]:
model.metrics_names

['loss', 'acc']